In [1]:
import torch
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
import torch.optim as optim
from training import train,test,train_resnet,test_resnet
import copy
from pca import PCADigitReducer

In [2]:

# Load MobileNetV3-Small pretrained on ImageNet
mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)
mobilenet_v3_small_pca = models.mobilenet_v3_small(pretrained=True)

c:\Users\micha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\micha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
import torch.nn as nn
# Modify the final layer for a custom number of classes (e.g., 10)
mobilenet_v3_small.classifier[3] = nn.Linear(in_features=1024, out_features=10)

In [10]:
n_epochs = 20
batch_size_train = 100
batch_size_test = 1000
learning_rate = 0.001
momentum = 0.5
log_interval = 10
num_clusters = 2

In [5]:
cifar10_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # https://pytorch.org/hub/pytorch_vision_resnet/
])

cifar10_train_loader = DataLoader(
    datasets.CIFAR10('/files/', train=True, download=True, transform=cifar10_transform),
    batch_size=batch_size_train, shuffle=True
)

cifar10_test_loader = DataLoader(
    datasets.CIFAR10('/files/', train=False, download=True, transform=cifar10_transform),
    batch_size=batch_size_test, shuffle=True
)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1]

In [7]:
train_loader_pca = copy.copy(cifar10_train_loader)
test_loader_pca = copy.copy(cifar10_test_loader)

train_loader_auto = copy.copy(cifar10_train_loader)
test_loader_auto = copy.copy(cifar10_test_loader)

In [11]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:  # Use your CIFAR-10 DataLoader here
    train_data.append(data.view(data.size(0), -1))  # Flatten images
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  # Combine all batches
train_labels = torch.cat(train_labels, dim=0)

# Convert to numpy for PCA
train_data_np = train_data.numpy()

# Perform PCA
n_components = 100  # Set the desired number of components
pca = PCADigitReducer(n_components)
train_data_reduced = pca.fit_transform(train_data_np)  # Reduce dimensions

# Reconstruct the dataset from the reduced dimensions
train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

# Reshape the reconstructed data back into the original image dimensions
train_data_reconstructed = train_data_reconstructed.view(-1, 3, 32, 32)

# Normalize the reconstructed dataset (use CIFAR-10 mean and std)
train_data_reconstructed = (train_data_reconstructed - torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)) / \
                           torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)

# Create a new DataLoader for the reconstructed data
batch_size_train = cifar10_train_loader.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

In [9]:
optimizer = optim.Adam(mobilenet_v3_small.parameters(), lr=learning_rate)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_resnet(epoch, mobilenet_v3_small, cifar10_train_loader, optimizer, log_interval, train_losses, train_counter)
    test_losses_classic_weak = []
    test_resnet(mobilenet_v3_small,cifar10_test_loader,test_losses_classic_weak)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.922065
Train Epoch: 1 [1000/50000 (2%)]	Loss: 2.335296
Train Epoch: 1 [2000/50000 (4%)]	Loss: 2.061989
Train Epoch: 1 [3000/50000 (6%)]	Loss: 2.082990
Train Epoch: 1 [4000/50000 (8%)]	Loss: 2.136349
Train Epoch: 1 [5000/50000 (10%)]	Loss: 1.793215
Train Epoch: 1 [6000/50000 (12%)]	Loss: 1.742800
Train Epoch: 1 [7000/50000 (14%)]	Loss: 1.626892
Train Epoch: 1 [8000/50000 (16%)]	Loss: 1.856075
Train Epoch: 1 [9000/50000 (18%)]	Loss: 1.646915
Train Epoch: 1 [10000/50000 (20%)]	Loss: 1.436072
Train Epoch: 1 [11000/50000 (22%)]	Loss: 1.713035
Train Epoch: 1 [12000/50000 (24%)]	Loss: 1.687623
Train Epoch: 1 [13000/50000 (26%)]	Loss: 1.573218
Train Epoch: 1 [14000/50000 (28%)]	Loss: 1.788940
Train Epoch: 1 [15000/50000 (30%)]	Loss: 1.681436
Train Epoch: 1 [16000/50000 (32%)]	Loss: 1.561346
Train Epoch: 1 [17000/50000 (34%)]	Loss: 1.425799
Train Epoch: 1 [18000/50000 (36%)]	Loss: 1.393291
Train Epoch: 1 [19000/50000 (38%)]	Loss: 1.375697
Train Epoch: 1 [20

In [10]:
test_losses_classic_weak = []
test_resnet(mobilenet_v3_small,cifar10_test_loader,test_losses_classic_weak)


Test set: Avg. loss: 0.8690, Accuracy: 7026/10000 (70%)



In [12]:
optimizer2 = optim.Adam(mobilenet_v3_small_pca.parameters(), lr=learning_rate)

train_losses2 = []
train_counter2 = []

for epoch in range(1, n_epochs + 1):  
    train_resnet(epoch, mobilenet_v3_small_pca, train_loader_reduced_pca, optimizer2, log_interval, train_losses2, train_counter2)
    test_losses_classic_weak = []
    test_resnet(mobilenet_v3_small_pca,cifar10_test_loader,test_losses_classic_weak)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.653326
Train Epoch: 1 [1000/50000 (2%)]	Loss: 0.886056
Train Epoch: 1 [2000/50000 (4%)]	Loss: 0.931995
Train Epoch: 1 [3000/50000 (6%)]	Loss: 1.055879
Train Epoch: 1 [4000/50000 (8%)]	Loss: 1.033132
Train Epoch: 1 [5000/50000 (10%)]	Loss: 0.820187
Train Epoch: 1 [6000/50000 (12%)]	Loss: 0.759605
Train Epoch: 1 [7000/50000 (14%)]	Loss: 1.088681
Train Epoch: 1 [8000/50000 (16%)]	Loss: 0.853769
Train Epoch: 1 [9000/50000 (18%)]	Loss: 1.298502
Train Epoch: 1 [10000/50000 (20%)]	Loss: 1.250193
Train Epoch: 1 [11000/50000 (22%)]	Loss: 1.017627
Train Epoch: 1 [12000/50000 (24%)]	Loss: 1.082289
Train Epoch: 1 [13000/50000 (26%)]	Loss: 0.886153
Train Epoch: 1 [14000/50000 (28%)]	Loss: 1.183026
Train Epoch: 1 [15000/50000 (30%)]	Loss: 0.861825
Train Epoch: 1 [16000/50000 (32%)]	Loss: 0.764934
Train Epoch: 1 [17000/50000 (34%)]	Loss: 0.799289
Train Epoch: 1 [18000/50000 (36%)]	Loss: 0.950717
Train Epoch: 1 [19000/50000 (38%)]	Loss: 0.827367
Train Epoch: 1 [20